In [ ]:
import os
import torch
from PIL import Image
from pathlib import Path
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import MllamaForConditionalGeneration, AutoProcessor

load_dotenv()

login(token=os.getenv("HF_TOKEN"))

In [ ]:
HF_MODEL_ID = "MykMaks/llama-3.2-11B-MM-20-MykMaks_da-wit-merged"
EVALUATION_IMAGES_DIR = "/home/ubuntu/meta_hackathon_oslo/evaluation/handwritten"
MAX_NEW_TOKENS = 1000

In [ ]:
model = MllamaForConditionalGeneration.from_pretrained(
    HF_MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(HF_MODEL_ID)

In [ ]:
def inference(image_url):
    image = Image.open(image_url)
    prompt = "<|image|><|begin_of_text|>Please summarise the contents of this image. Present as markdown."

    inputs = processor(image, prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)

    return processor\
        .decode(output[0])\
        .replace(prompt, "")\
        .replace("<|begin_of_text|>", "")

In [5]:
directory = Path(EVALUATION_IMAGES_DIR)

for image_path in directory.iterdir():
    inference_result = inference(image_path)
    print(inference_result)

 Tabell 17 - Oversikt over estimatusikkerhet og usikkerhetsdrivere for A4

Kostnadspost

Dagsone Narvik-Tromsø

Tunnel Narvik-Tromsø

Bru Narvik-Tromsø

Jernbaneteknikk Narvik-Tromsø

Otfotbanen

Stasjoner

Godsterminaler + Verksted/Serviceanlegg

Omnformerstasjon

Felleskostnader entreprenør

Planlegging og prosjektering

Byggherreorganisasjon

Grunnervær

Basisestimat

Estimatusikkerheten gir en forventet kostnad (83 923 mill. kr) fra basisestimatet (80 436 mill. kr) som trippelanslagene til usikkerhetsdriverne virker på.

|      | Basisestimat   | Basisestimat   | Basisestimat   | M %   | M      | Høy %   | Høy      |
|------|----------------|----------------|----------------|-------|--------|---------|----------|
| Kostnadspost | 7 224          | -11 %          | 6 444          | 0 %   | 7 224   | 25 %     | 9 044    |
| Dagsone Narvik-Tromsø | 25 329        | -7 %           | 23 506        | 0 %   | 25 329  | 29 %     | 32 624   |
| Tunnel Narvik-Tromsø | 3 197         | -24 %    